## 1. The operations team wants to ensure the integrity of the events data by removing duplicates. Check for and remove duplicate rows in the events dataframe based on store_id, campaign_id, and product_code. How many duplicate rows were removed?

In [715]:
import pandas as pd

In [716]:
df = pd.read_csv("fact_events.csv")

In [717]:
df.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272


In [718]:
columns_to_match = ["store_id", "campaign_id", "product_code"]

In [719]:
df_to_match = df[columns_to_match]

In [720]:
df_to_match.head()

,store_id,campaign_id,product_code
0,STCHE-1,CAMP_DIW_01,P04
1,STCHE-3,CAMP_SAN_01,P04
2,STBLR-9,CAMP_DIW_01,P02
3,STBLR-7,CAMP_DIW_01,P05
4,STHYD-5,CAMP_SAN_01,P15


In [721]:
rows_original = df_to_match.shape[0]

In [722]:
rows_original

1510

In [723]:
df_to_match.duplicated().sum()

10

In [724]:
dup_bool = df_to_match.duplicated()
dup_indices = df_to_match[dup_bool].index

In [725]:
dup_bool

0       False
1       False
2       False
3       False
4       False
        ...  
1505    False
1506    False
1507    False
1508    False
1509    False
Length: 1510, dtype: bool

In [726]:
dup_indices

Index([74, 410, 541, 759, 922, 1003, 1155, 1227, 1313, 1401], dtype='int64')

In [727]:
df_no_duplicates = df.drop(index=dup_indices)

In [728]:
df_no_duplicates.shape

(1500, 9)

## Total number of rows removed are 1510 - 1500 = 10

## 2. How many cities have more than 5 stores?

In [729]:
df_cities = pd.read_csv("dim_stores.csv")

In [730]:
df_cities.head()

,store_id,city
0,STTRV-0,Trivandrum
1,STMDU-3,Madurai
2,STHYD-6,Hyderabad
3,STVSK-1,Visakhapatnam
4,STCBE-3,Coimbatore


In [731]:
df_cities.groupby("city").count()

,store_id
city,
Bengaluru,10
Chennai,8
Coimbatore,5
Hyderabad,7
Madurai,4
Mangalore,3
Mysuru,4
Trivandrum,2
Vijayawada,2


In [732]:
df_cities.groupby("city").count()

,store_id
city,
Bengaluru,10
Chennai,8
Coimbatore,5
Hyderabad,7
Madurai,4
Mangalore,3
Mysuru,4
Trivandrum,2
Vijayawada,2


In [733]:
(df_cities.groupby("city").size() > 5).sum()

3

## 3 cities have more than 5 stores

## 3. The sales team has identified missing values in the quantity_sold(before_promo) data. Estimate these values using the median quantity sold before the promotion. How many missing values were filled, and what is the median used for imputation?

In [734]:
df_no_duplicates.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272


In [735]:
df_no_duplicates["quantity_sold(before_promo)"].isnull().sum()

20

In [736]:
median_reqd = df_no_duplicates["quantity_sold(before_promo)"].median()

In [737]:
median_reqd

78.0

In [738]:
df_no_duplicates['quantity_sold(before_promo)'] = (
    df_no_duplicates['quantity_sold(before_promo)'].fillna(median_reqd)
)


In [739]:
df_no_duplicates['quantity_sold(before_promo)'].isnull().sum()

0

## 20 missing values were filled with median = 78.0

In [740]:
df_no_duplicates["product_code"].nunique()

15

## 4. Identify the product category with the lowest base price before the promotion.

In [741]:
df_products = pd.read_csv("dim_products.csv")

In [742]:
df_products.head()

,product_code,product_name,category
0,P01,Atliq_Masoor_Dal (1KG),Grocery & Staples
1,P02,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples
2,P03,Atliq_Suflower_Oil (1L),Grocery & Staples
3,P04,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
4,P05,Atliq_Scrub_Sponge_For_Dishwash,Home Care


In [743]:
df_products["product_code"].nunique()

15

In [744]:
df_no_duplicates.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272


In [745]:
df_new = pd.merge(df_no_duplicates, df_products, on="product_code")

In [746]:
min_prices = df_new.groupby("category")["base_price(before_promo)"].min()

In [747]:
min_prices

category
Combo1               3000
Grocery & Staples     156
Home Appliances       350
Home Care              55
Personal Care          50
Name: base_price(before_promo), dtype: int64

In [748]:
lowest_product_category = min_prices.idxmin()

In [749]:
lowest_product_category

'Personal Care'

## Personal Care is product category with the lowest base price before the promotion

## 5. What is the total quantity sold after the promotion for the BOGOF promo type during the Diwali campaign?

In [750]:
df_no_duplicates.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo)
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272


In [751]:
df_campaigns = pd.read_csv("dim_campaigns.csv")

In [752]:
df_campaigns.head()

,campaign_id,campaign_name,start_date,end_date
0,CAMP_DIW_01,Diwali,12/11/2023,18-11-2023
1,CAMP_SAN_01,Sankranti,10/1/2024,16-01-2024


In [753]:
df_new_2 = pd.merge(df_new,df_campaigns, on="campaign_id")

In [754]:
df_new_2.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),product_name,category,campaign_name,start_date,end_date
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Sankranti,10/1/2024,16-01-2024
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Atliq_Scrub_Sponge_For_Dishwash,Home Care,Diwali,12/11/2023,18-11-2023
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Atliq_Home_Essential_8_Product_Combo,Combo1,Sankranti,10/1/2024,16-01-2024


In [755]:
quantity_sold_after = (
    df_new_2[df_new_2["campaign_name"] == "Diwali"]
    .groupby("promo_type")["quantity_sold(after_promo)"]
    .sum()
)


In [756]:
quantity_sold_after["BOGOF"]

34461

### 34461.0 is the total quantity sold after the promotion for the BOGOF promo type during the Diwali campaign

## 6. Which store recorded the highest quantity sold after the promotion during the Diwali campaign?

In [757]:
df_new_2.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),product_name,category,campaign_name,start_date,end_date
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Sankranti,10/1/2024,16-01-2024
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Atliq_Scrub_Sponge_For_Dishwash,Home Care,Diwali,12/11/2023,18-11-2023
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Atliq_Home_Essential_8_Product_Combo,Combo1,Sankranti,10/1/2024,16-01-2024


In [758]:
df_stores = pd.read_csv("dim_stores.csv")

In [759]:
df_stores.head()

,store_id,city
0,STTRV-0,Trivandrum
1,STMDU-3,Madurai
2,STHYD-6,Hyderabad
3,STVSK-1,Visakhapatnam
4,STCBE-3,Coimbatore


In [760]:
stores = df_new_2[df_new_2["campaign_name"]=="Diwali"].groupby("store_id")["quantity_sold(after_promo)"].sum()

In [761]:
store_rqd= stores.idxmax()

In [762]:
store_rqd

'STCHE-4'

## STCHE-4 recorded the highest quantity sold after the promotion during the Diwali campaign

## 7. Understand which campaigns had the most successful outcomes. Compare the total quantities sold before and after the promotions for the Sankranti and Diwali campaigns. Which campaign saw a greater increase in sales?

In [763]:
diwali_increase = df_new_2[df_new_2["campaign_name"]=="Diwali"]["quantity_sold(after_promo)"].sum() - df_new_2[df_new_2["campaign_name"]=="Diwali"]["quantity_sold(before_promo)"].sum()  

In [764]:
diwali_increase

73648.0

In [765]:
sankranti_increase = df_new_2[df_new_2["campaign_name"]=="Sankranti"]["quantity_sold(after_promo)"].sum() - df_new_2[df_new_2["campaign_name"]=="Sankranti"]["quantity_sold(before_promo)"].sum()  

In [766]:
sankranti_increase

154175.0

## Above result shows that Sankranti campaign saw a greater increase in sales

## 8. Which product recorded the highest Incremental Revenue Percentage (IR%) during the Sankranti campaign? What is the IR% for this product?

In [767]:
df_new_2.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),product_name,category,campaign_name,start_date,end_date
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Sankranti,10/1/2024,16-01-2024
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Atliq_Scrub_Sponge_For_Dishwash,Home Care,Diwali,12/11/2023,18-11-2023
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Atliq_Home_Essential_8_Product_Combo,Combo1,Sankranti,10/1/2024,16-01-2024


In [768]:
df_new_2["revenue_before_promo"] = df_new_2["base_price(before_promo)"]*df_new_2["quantity_sold(before_promo)"]
df_new_2["revenue_after_promo"] = df_new_2["base_price(after_promo)"]*df_new_2["quantity_sold(after_promo)"]

In [769]:
df_new_2.duplicated().sum()

0

In [770]:
df_new_2.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),product_name,category,campaign_name,start_date,end_date,revenue_before_promo,revenue_after_promo
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023,94830.0,62279
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Sankranti,10/1/2024,16-01-2024,140230.0,300070
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023,289820.0,281088
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Atliq_Scrub_Sponge_For_Dishwash,Home Care,Diwali,12/11/2023,18-11-2023,6710.0,4387
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Atliq_Home_Essential_8_Product_Combo,Combo1,Sankranti,10/1/2024,16-01-2024,366000.0,680000


In [771]:
sankranti_ir = (
    df_new_2[df_new_2["campaign_name"] == "Sankranti"]
    .groupby("product_name")[["revenue_before_promo", "revenue_after_promo"]]
    .sum()
)

In [772]:
sankranti_ir["IR%"] = ((sankranti_ir["revenue_after_promo"] - sankranti_ir["revenue_before_promo"]) / sankranti_ir["revenue_before_promo"]) * 100

In [773]:
sankranti_ir

,revenue_before_promo,revenue_after_promo,IR%
product_name,,,
Atliq_Body_Milk_Nourishing_Lotion (120ML),239040.0,146462,-38.729083
Atliq_Cream_Beauty_Bathing_Soap (125GM),75400.0,42328,-43.862069
Atliq_Curtains,583500.0,1065450,82.596401
Atliq_Doodh_Kesar_Body_Lotion (200ML),373350.0,258970,-30.636132
Atliq_Double_Bedsheet_set,2485910.0,4649330,87.027286
Atliq_Farm_Chakki_Atta (1KG),6813550.0,12779800,87.564485
Atliq_Fusion_Container_Set_of_3,561080.0,342722,-38.917445
Atliq_High_Glo_15W_LED_Bulb,1740550.0,3303125,89.774784
Atliq_Home_Essential_8_Product_Combo,16185000.0,31027500,91.705283


In [774]:
top_product = sankranti_ir["IR%"].idxmax()
top_ir = sankranti_ir["IR%"].max()

In [775]:
top_product, top_ir

('Atliq_Suflower_Oil (1L)', 91.826561324304)

## Atliq_Suflower_Oil (1L) recorded the highest Incremental Revenue Percentage (IR%) during the Sankranti campaign with IR% as 91.82

## 9. Which store in Visakhapatnam recorded the lowest Incremental Sold Units Percentage (ISU%) during the Diwali campaign? What is the ISU% for that store?

In [776]:
df_new_2 = pd.merge(df_new_2,df_cities, on="store_id")

In [777]:
df_new_2.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),product_name,category,campaign_name,start_date,end_date,revenue_before_promo,revenue_after_promo,city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023,94830.0,62279,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Sankranti,10/1/2024,16-01-2024,140230.0,300070,Chennai
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023,289820.0,281088,Bengaluru
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Atliq_Scrub_Sponge_For_Dishwash,Home Care,Diwali,12/11/2023,18-11-2023,6710.0,4387,Bengaluru
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Atliq_Home_Essential_8_Product_Combo,Combo1,Sankranti,10/1/2024,16-01-2024,366000.0,680000,Hyderabad


In [778]:
diwali_visak_isu = (
    df_new_2[(df_new_2["campaign_name"] == "Diwali") & (df_new_2["city"]=="Visakhapatnam")]
    .groupby("store_id")[["quantity_sold(before_promo)", "quantity_sold(after_promo)"]]
    .sum()
)

In [779]:
diwali_visak_isu

,quantity_sold(before_promo),quantity_sold(after_promo)
store_id,,
STVSK-0,1768.0,3005
STVSK-1,1903.0,3078
STVSK-2,1701.0,2860
STVSK-3,1780.0,2656
STVSK-4,1926.0,2908


In [780]:
diwali_visak_isu["ISU%"] = (diwali_visak_isu["quantity_sold(after_promo)"] - diwali_visak_isu["quantity_sold(before_promo)"])/diwali_visak_isu["quantity_sold(before_promo)"] 

In [781]:
diwali_visak_isu

,quantity_sold(before_promo),quantity_sold(after_promo),ISU%
store_id,,,
STVSK-0,1768.0,3005,0.699661
STVSK-1,1903.0,3078,0.617446
STVSK-2,1701.0,2860,0.681364
STVSK-3,1780.0,2656,0.492135
STVSK-4,1926.0,2908,0.509865


In [782]:
diwali_visak_isu_min_store = diwali_visak_isu["ISU%"].idxmin()
diwali_visak_isu_min = diwali_visak_isu["ISU%"].min()


In [783]:
diwali_visak_isu_min_store, diwali_visak_isu_min

('STVSK-3', 0.49213483146067416)

## Store STVSK-3 recorded lowest ISU% of 49.21

## 10. Which promo type had both a negative Incremental Revenue Percentage (IR%) and Incremental Sold Units Percentage (ISU%) during the Sankranti campaign?

In [784]:
df_new_2.head()

,event_id,store_id,campaign_id,product_code,base_price(before_promo),quantity_sold(before_promo),promo_type,base_price(after_promo),quantity_sold(after_promo),product_name,category,campaign_name,start_date,end_date,revenue_before_promo,revenue_after_promo,city
0,8481be,STCHE-1,CAMP_DIW_01,P04,290,327.0,25% OFF,217,287,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023,94830.0,62279,Chennai
1,20618e,STCHE-3,CAMP_SAN_01,P04,370,379.0,BOGOF,185,1622,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,Sankranti,10/1/2024,16-01-2024,140230.0,300070,Chennai
2,f30579,STBLR-9,CAMP_DIW_01,P02,860,337.0,33% OFF,576,488,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,Diwali,12/11/2023,18-11-2023,289820.0,281088,Bengaluru
3,4f570c,STBLR-7,CAMP_DIW_01,P05,55,122.0,25% OFF,41,107,Atliq_Scrub_Sponge_For_Dishwash,Home Care,Diwali,12/11/2023,18-11-2023,6710.0,4387,Bengaluru
4,6d153f,STHYD-5,CAMP_SAN_01,P15,3000,122.0,500 Cashback,2500,272,Atliq_Home_Essential_8_Product_Combo,Combo1,Sankranti,10/1/2024,16-01-2024,366000.0,680000,Hyderabad


In [785]:
promo_ir_isu_sankranti = df_new_2[df_new_2["campaign_name"]=="Sankranti"].groupby("promo_type")[["revenue_before_promo","revenue_after_promo","quantity_sold(before_promo)","quantity_sold(after_promo)"]].sum()

In [786]:
promo_ir_isu_sankranti["IR%"] = 100*((promo_ir_isu_sankranti["revenue_after_promo"] - promo_ir_isu_sankranti["revenue_before_promo"])
                                     /promo_ir_isu_sankranti["revenue_before_promo"])
promo_ir_isu_sankranti["ISU%"] = 100*((promo_ir_isu_sankranti["quantity_sold(after_promo)"] - promo_ir_isu_sankranti["quantity_sold(before_promo)"])
                                     /promo_ir_isu_sankranti["quantity_sold(before_promo)"])

In [787]:
promo_ir_isu_sankranti

,revenue_before_promo,revenue_after_promo,quantity_sold(before_promo),quantity_sold(after_promo),IR%,ISU%
promo_type,,,,,,
25% OFF,935195.0,567387,6601.0,5307,-39.329552,-19.603090
33% OFF,20483136.0,19273955,33624.0,47459,-5.903300,41.146205
50% OFF,527482.0,363564,4451.0,6100,-31.075563,37.047854
500 Cashback,16185000.0,31027500,5395.0,12411,91.705283,130.046339
BOGOF,19355170.0,36451055,47823.0,180792,88.327227,278.044037


In [795]:
promo_rqd = promo_ir_isu_sankranti[(promo_ir_isu_sankranti["IR%"] < 0) & (promo_ir_isu_sankranti["ISU%"] < 0)]

In [796]:
promo_rqd.index, promo_rqd["IR%"], promo_rqd["ISU%"]

(Index(['25% OFF'], dtype='object', name='promo_type'),
 promo_type
 25% OFF   -39.329552
 Name: IR%, dtype: float64,
 promo_type
 25% OFF   -19.60309
 Name: ISU%, dtype: float64)

## '25% OFF' promo type had both a negative Incremental Revenue Percentage (IR%) and Incremental Sold Units Percentage (ISU%) during the Sankranti campaign